In [4]:
from nba_api.stats.static import teams
from nba_api.stats.static import players
import pandas as pd
import numpy as np
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import leaguedashplayershotlocations
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date, timedelta
import datetime
import time, os

In [5]:
pd.options.display.max_columns = None

In [6]:
from nba_api.stats.endpoints import TeamDetails
nba_teams = teams.get_teams()
nba_players = players.get_active_players()

In [7]:
nba_df = pd.DataFrame.from_dict(nba_teams)
nba_pl = pd.DataFrame.from_dict(nba_players)

In [8]:
nba_abbr = nba_df['abbreviation'].unique().tolist()

In [9]:
nba_id = nba_df['id'].unique().tolist()

In [10]:
pl_id = nba_pl['id'].unique().tolist()

In [11]:
nba_pl

,id,full_name,first_name,last_name,is_active
0,1630173,Precious Achiuwa,Precious,Achiuwa,True
1,203500,Steven Adams,Steven,Adams,True
2,1628389,Bam Adebayo,Bam,Adebayo,True
3,1630534,Ochai Agbaji,Ochai,Agbaji,True
4,1630583,Santi Aldama,Santi,Aldama,True
...,...,...,...,...,...
526,201152,Thaddeus Young,Thaddeus,Young,True
527,1629027,Trae Young,Trae,Young,True
528,1630209,Omer Yurtseven,Omer,Yurtseven,True
529,203469,Cody Zeller,Cody,Zeller,True


In [12]:
nba_df

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [13]:
'''from gspread_pandas import Spread, Client
spread = Spread('NBA_stats')
spread.df_to_sheet(nba_pl, index=False,sheet='NBA_PL',replace=True)'''

"from gspread_pandas import Spread, Client\nspread = Spread('NBA_stats')\nspread.df_to_sheet(nba_pl, index=False,sheet='NBA_PL',replace=True)"

In [14]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
rosters = []

for i in nba_id:
    roster = commonteamroster.CommonTeamRoster(season="2021-22", team_id=i) 
    r = roster.get_normalized_dict()
    tms = r['CommonTeamRoster']
    df = pd.DataFrame.from_dict(tms)
    #a = df['GAME_ID'].unique().tolist()
    rosters.append(df)


In [15]:
nbaroster22 = pd.concat(rosters)

In [16]:
nba_id = nbaroster22[['TeamID','PLAYER_ID','PLAYER']]

In [18]:
nbaroster22

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612737,2021,00,Delon Wright,Delon,delon-wright,0,G,6-5,185,"APR 26, 1992",30.0,6,Utah,1626153,None
1,1610612737,2021,00,Jalen Johnson,Jalen,jalen-johnson,1,F,6-8,219,"DEC 18, 2001",20.0,R,Duke,1630552,#20 Pick in 2021 Draft
2,1610612737,2021,00,Sharife Cooper,Sharife,sharife-cooper,2,G,6-1,176,"JUN 11, 2001",21.0,R,Auburn,1630536,None
3,1610612737,2021,00,Kevin Huerter,Kevin,kevin-huerter,3,G-F,6-7,198,"AUG 27, 1998",23.0,3,Maryland,1628989,None
4,1610612737,2021,00,Skylar Mays,Skylar,skylar-mays,4,G,6-4,205,"SEP 05, 1997",24.0,1,Louisiana State,1630219,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,1610612766,2021,00,JT Thor,JT,jt-thor,21,F,6-9,203,"AUG 26, 2002",19.0,R,Auburn,1630550,Draft Rights Traded from DET on 08/06/21
13,1610612766,2021,00,Kai Jones,Kai,kai-jones,23,C-F,6-10,221,"JAN 19, 2001",21.0,R,Texas,1630539,None
14,1610612766,2021,00,Mason Plumlee,Mason,mason-plumlee,24,F-C,6-11,254,"MAR 05, 1990",32.0,8,Duke,203486,None
15,1610612766,2021,00,P.J. Washington,P.J.,pj-washington,25,F,6-7,230,"AUG 23, 1998",23.0,2,Kentucky,1629023,None


In [19]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import playerdashptpass
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
rosters = []

for i in nba_id.iterrows():
    time.sleep(1)
    roster = playerdashptpass.PlayerDashPtPass(season="2021-22", team_id=i[0], player_id=i[1]) 
    r = roster.get_normalized_dict()
    tms = r['PassesMade']
    df = pd.DataFrame.from_dict(tms)
    #a = df['GAME_ID'].unique().tolist()
    rosters.append(df)

In [20]:
passes = pd.concat(rosters)

{'PassesMade': [], 'PassesReceived': []}

In [64]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import playerdashboardbyshootingsplits
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType




#roster = playerdashboardbyshootingsplits.PlayerDashboardByShootingSplits(season="2021-22",player_id=1629023) 
r = roster.get_normalized_dict()
tms = r['ShotAreaPlayerDashboard']
df = pd.DataFrame.from_dict(tms)
#a = df['GAME_ID'].unique().tolist()
#rosters.append(df)

In [65]:
df

,GROUP_SET,GROUP_VALUE,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,EFG_PCT,BLKA,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,EFG_PCT_RANK,BLKA_RANK,PCT_AST_2PM_RANK,PCT_UAST_2PM_RANK,PCT_AST_3PM_RANK,PCT_UAST_3PM_RANK,PCT_AST_FGM_RANK,PCT_UAST_FGM_RANK,CFID,CFPARAMS
0,Shot Area,Restricted Area,116,173,0.671,0,0,0.000,0.671,17,0.767,0.233,0.000,0.000,0.767,0.233,1,2,1,4,5,4,2,7,1,1,4,2,1,1,49,Restricted Area
1,Shot Area,In The Paint (Non-RA),22,51,0.431,0,0,0.000,0.431,5,0.909,0.091,0.000,0.000,0.909,0.091,3,3,3,4,5,4,4,6,2,2,4,2,3,3,49,In The Paint (Non-RA)
2,Shot Area,Mid-Range,2,7,0.286,0,0,0.000,0.286,1,1.000,0.000,0.000,0.000,1.000,0.000,6,6,6,4,5,4,6,3,3,3,4,2,6,4,49,Mid-Range
3,Shot Area,Left Corner 3,15,36,0.417,15,36,0.417,0.625,1,0.000,0.000,1.000,0.000,1.000,0.000,4,5,4,2,3,2,3,3,4,3,2,2,4,4,49,Left Corner 3
4,Shot Area,Right Corner 3,9,41,0.220,9,41,0.220,0.329,0,0.000,0.000,1.000,0.000,1.000,0.000,5,4,5,3,2,3,5,1,4,3,3,2,5,4,49,Right Corner 3
5,Shot Area,Above the Break 3,86,221,0.389,86,221,0.389,0.584,1,0.000,0.000,0.965,0.035,0.965,0.035,2,1,2,1,1,1,1,3,4,3,1,1,1,2,49,Above the Break 3
6,Shot Area,Backcourt,0,3,0.000,0,3,0.000,0.000,0,0.000,0.000,0.000,0.000,0.000,0.000,7,7,7,4,4,4,7,1,4,3,4,2,7,4,49,Backcourt


In [45]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import leaguehustlestatsplayer
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType




roster = leaguehustlestatsplayer.LeagueHustleStatsPlayer(season="2021-22") 
r = roster.get_normalized_dict()
tms = r['HustleStatsPlayer']
df = pd.DataFrame.from_dict(tms)
#a = df['GAME_ID'].unique().tolist()
#rosters.append(df)

In [46]:
df

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,G,MIN,CONTESTED_SHOTS,CONTESTED_SHOTS_2PT,CONTESTED_SHOTS_3PT,DEFLECTIONS,CHARGES_DRAWN,SCREEN_ASSISTS,SCREEN_AST_PTS,OFF_LOOSE_BALLS_RECOVERED,DEF_LOOSE_BALLS_RECOVERED,LOOSE_BALLS_RECOVERED,PCT_LOOSE_BALLS_RECOVERED_OFF,PCT_LOOSE_BALLS_RECOVERED_DEF,OFF_BOXOUTS,DEF_BOXOUTS,BOX_OUT_PLAYER_TEAM_REBS,BOX_OUT_PLAYER_REBS,BOX_OUTS,PCT_BOX_OUTS_OFF,PCT_BOX_OUTS_DEF,PCT_BOX_OUTS_TEAM_REB,PCT_BOX_OUTS_REB
0,203932,Aaron Gordon,1610612743,DEN,26.0,75,2375.0,399,260,139,68,1,53,130,31,19,50,0.620,0.380,10,29,33,24,39,0.256,0.744,0.943,0.686
1,1630565,Aaron Henry,1610612755,PHI,22.0,6,17.0,5,3,2,1,0,0,0,0,0,0,0.000,0.000,0,1,0,0,1,0.000,1.000,0.000,0.000
2,1628988,Aaron Holiday,1610612756,PHX,25.0,63,1021.0,166,76,90,83,1,8,21,12,16,28,0.429,0.571,2,18,19,3,20,0.100,0.900,0.950,0.150
3,1630174,Aaron Nesmith,1610612738,BOS,22.0,52,574.0,151,95,56,29,4,4,10,5,8,13,0.385,0.615,3,23,23,10,26,0.115,0.885,0.958,0.417
4,1630598,Aaron Wiggins,1610612760,OKC,23.0,50,1209.0,302,153,149,65,0,14,30,10,16,26,0.385,0.615,1,23,22,9,24,0.042,0.958,0.957,0.391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,203897,Zach LaVine,1610612741,CHI,27.0,67,2328.0,398,169,229,69,1,30,63,26,16,42,0.619,0.381,3,25,27,14,28,0.107,0.893,0.964,0.500
601,1630285,Zavier Simpson,1610612760,OKC,25.0,4,174.0,39,15,24,9,0,0,0,1,3,4,0.250,0.750,0,1,1,0,1,0.000,1.000,1.000,0.000
602,1630192,Zeke Nnaji,1610612743,DEN,21.0,41,698.0,151,82,69,31,0,39,97,10,6,16,0.625,0.375,22,30,47,31,52,0.423,0.577,0.979,0.646
603,1630533,Ziaire Williams,1610612763,MEM,20.0,62,1346.0,287,133,154,61,0,7,15,8,11,19,0.421,0.579,0,14,12,3,14,0.000,1.000,1.000,0.250


In [13]:

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
         'stats-project-354002-ff7285022b56.json', scope) # Your json file here

gc = gspread.authorize(credentials)

sh = gc.open("NBA_stats")

worksheet = sh.worksheet('NBA_gamelogs')

data = worksheet.get_all_values()
headers = data.pop(0)

gl = pd.DataFrame(data, columns=headers)

In [14]:
gl['GAME_DATE']= pd.to_datetime(gl['GAME_DATE']).dt.strftime('%Y-%m-%d')
old_dates = gl['GAME_DATE'].unique().tolist()
old_dates.sort()

In [15]:
datetime.datetime.strptime(old_dates[-1], "%Y-%m-%d")

datetime.datetime(2022, 10, 31, 0, 0)

In [16]:

start_date = (datetime.datetime.strptime(old_dates[-1], "%Y-%m-%d")+timedelta(days = 1)).strftime("%m/%d/%Y")

In [17]:
start_date

'11/01/2022'

In [18]:
yest_date = (date.today()- timedelta(days = 1)).strftime("%m/%d/%Y")

In [19]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import playergamelogs
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType


s = playergamelogs.PlayerGameLogs(season_nullable='2022-23', date_from_nullable=start_date, date_to_nullable=yest_date).get_normalized_dict()
games = s['PlayerGameLogs']
gl_add = pd.DataFrame.from_dict(games)

In [20]:
gl_add['opp_abbr'] = gl_add['MATCHUP'].str[-3:]

In [21]:
nba_rosters.rename(columns={'TeamID':'TEAM_ID','PLAYER':'PLAYER_NAME'},inplace=True)

/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/2198788671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_rosters.rename(columns={'TeamID':'TEAM_ID','PLAYER':'PLAYER_NAME'},inplace=True)


In [22]:
nbagamelogs = pd.merge(gl_add, nba_rosters, on=['TEAM_ID','PLAYER_ID','PLAYER_NAME'],how='left')

In [23]:
gls= nbagamelogs[['GAME_DATE','GAME_ID','PLAYER_ID','PLAYER_NAME','POSITION','HEIGHT','WEIGHT','TEAM_ID','TEAM_NAME','opp_abbr','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA','PTS','REB','AST','TOV','STL','BLK']]

In [24]:
gl_con=[gl,gls]

In [25]:
gamelogs = pd.concat(gl_con)

In [26]:
gamelogs['GAME_DATE']= pd.to_datetime(gamelogs['GAME_DATE']).dt.strftime('%m/%d/%Y')

In [27]:
gamelogs

,GAME_DATE,GAME_ID,PLAYER_ID,PLAYER_NAME,POSITION,HEIGHT,WEIGHT,TEAM_ID,TEAM_NAME,opp_abbr,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,PTS,REB,AST,TOV,STL,BLK,PTS_RANK,FGA_RANK,FG3A_RANK,FTA_RANK,REB_RANK,AST_RANK
0,10/28/2022,22200073,1630178,Tyrese Maxey,G,6/2/2022,200,1610612755,Philadelphia 76ers,TOR,39.81833333,15,20,9,12,5,6,44,8,4,2,1,0,1,33,8,64,82,110
1,10/30/2022,22200090,1628378,Donovan Mitchell,G,6/1/2022,215,1610612739,Cleveland Cavaliers,NYK,37.16166667,12,20,8,13,6,8,38,3,12,4,1,2,6,33,4,30,345,3
2,10/30/2022,22200090,201567,Kevin Love,F-C,6/8/2022,251,1610612739,Cleveland Cavaliers,NYK,22.47,9,14,8,13,3,3,29,8,1,1,0,0,38,122,4,189,82,356
3,10/30/2022,22200092,203926,Doug McDermott,F,6/6/2022,225,1610612759,San Antonio Spurs,MIN,25.42333333,8,16,7,14,0,0,23,3,1,1,0,1,70,81,2,408,345,356
4,10/29/2022,22200081,1628989,Kevin Huerter,G-F,6/7/2022,198,1610612758,Sacramento Kings,MIA,38.18,8,11,7,8,4,6,27,3,7,0,0,0,45,191,46,64,345,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,11/01/2022,0022200105,1630532,Franz Wagner,F,6-10,220,1610612753,Orlando Magic,OKC,36.716667,8,18,1,5,3,3,20,1,7,2,1,2,54,29,73,79,271,22
350,11/01/2022,0022200105,1629626,Bol Bol,C-F,7-2,220,1610612753,Orlando Magic,OKC,30.883333,4,6,1,1,4,4,13,12,1,6,0,4,111,193,236,49,12,197
351,11/01/2022,0022200105,1630197,Aleksej Pokusevski,F,7-0,210,1610612760,Oklahoma City Thunder,ORL,31.266667,7,13,2,5,0,0,16,9,1,2,2,3,81,70,73,192,33,197
352,11/01/2022,0022200105,1628976,Wendell Carter Jr.,C-F,6-10,270,1610612753,Orlando Magic,OKC,35.433333,10,15,2,2,8,8,30,12,2,2,2,1,10,58,193,14,12,136


In [28]:
from gspread_pandas import Spread, Client
spread = Spread('NBA_stats')
spread.df_to_sheet(gamelogs, index=False,sheet='NBA_gamelogs',replace=True)

In [121]:
#nba_id

In [94]:
#leaguedashplayerstats.LeagueDashPlayerStats(season=2022, )

In [43]:
'''# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import playerdashptpass
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
passing = []

for i in nba_rosters.iterrows():
    roster = playerdashptpass.PlayerDashPtPass(season="2022-23", player_id = i[1][2], team_id=i[1][0]) 
    r = roster.get_normalized_dict()
    tms = r['PassesMade']
    df = pd.DataFrame.from_dict(tms)
    #a = df['GAME_ID'].unique().tolist()
    passing.append(df)'''


'# Query for the last regular season game where the Pacers were playing\nfrom nba_api.stats.endpoints import playerdashptpass\nfrom nba_api.stats.library.parameters import Season\nfrom nba_api.stats.library.parameters import SeasonType\npassing = []\n\nfor i in nba_rosters.iterrows():\n    roster = playerdashptpass.PlayerDashPtPass(season="2022-23", player_id = i[1][2], team_id=i[1][0]) \n    r = roster.get_normalized_dict()\n    tms = r[\'PassesMade\']\n    df = pd.DataFrame.from_dict(tms)\n    #a = df[\'GAME_ID\'].unique().tolist()\n    passing.append(df)'

In [ ]:
#passing22 = pd.concat(passing)

In [97]:
nbaroster22

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612737,2022,00,Frank Kaminsky,Frank,frank-kaminsky,0,F-C,7-0,240,"APR 04, 1993",29.0,7,Wisconsin,1626163,Signed on 07/11/22
1,1610612737,2022,00,Jalen Johnson,Jalen,jalen-johnson,1,F,6-8,219,"DEC 18, 2001",20.0,1,Duke,1630552,#20 Pick in 2021 Draft
2,1610612737,2022,00,Trent Forrest,Trent,trent-forrest,2,G,6-4,210,"JUN 12, 1998",24.0,2,Florida State,1630235,Signed on 08/08/22
3,1610612737,2022,00,Aaron Holiday,Aaron,aaron-holiday,3,G,6-0,185,"SEP 30, 1996",26.0,4,UCLA,1628988,Signed on 07/06/22
4,1610612737,2022,00,Dejounte Murray,Dejounte,dejounte-murray,5,G,6-5,180,"SEP 19, 1996",26.0,5,Washington,1627749,Traded from SAS on 06/30/22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,1610612766,2022,00,Gordon Hayward,Gordon,gordon-hayward,20,F,6-7,225,"MAR 23, 1990",32.0,12,Butler,202330,Traded from BOS on 11/29/20
12,1610612766,2022,00,JT Thor,JT,jt-thor,21,F,6-9,203,"AUG 26, 2002",20.0,1,Auburn,1630550,Draft Rights Traded from DET on 08/06/21
13,1610612766,2022,00,Kai Jones,Kai,kai-jones,23,C-F,6-11,221,"JAN 19, 2001",21.0,1,Texas,1630539,Draft Rights Traded from NYK on 07/30/21
14,1610612766,2022,00,Mason Plumlee,Mason,mason-plumlee,24,F-C,7-0,254,"MAR 05, 1990",32.0,9,Duke,203486,Traded from DET on 08/06/21


In [98]:
nba_rosters = nbaroster22[['TeamID','SEASON','PLAYER_ID','PLAYER','POSITION','HEIGHT','WEIGHT']]

In [99]:
for i in nba_rosters.iterrows():
    print(i[1][2])
    

1626163
1630552
1630235
1628988
1627749
1629633
203200
1629027
1629631
203992
1631100
203991
1630168
1628381
1631213
1630249
1628369
203943
1627759
201568
1628401
1630202
1629684
1627763
1631120
1629662
1630573
203935
1628436
201143
1629057
1628382
201933
201567
1627747
1630596
1630205
1629685
1629636
201937
1630600
1626224
203526
1629603
1628386
1629731
201577
1630171
1628378
1630604
1631220
1629627
203468
1628984
1630529
1630230
1626195
1629637
1630700
1630184
1627742
1630631
202685
1626204
1630530
202066
1629632
1628366
203083
1627884
1627936
201609
203897
202696
201942
1630245
1628396
1630792
1630250
1629750
1631207
1628961
1630172
201580
1629033
1630267
1630702
203939
1630182
1627827
203501
1628373
1628416
1630593
203493
203915
1626174
1628467
202722
1629029
1631128
1629008
1630538
202397
203484
201599
1631212
1628432
1631298
1628971
203999
1631221
1630192
1627750
1628427
201145
203932
1630228
1628978
203210
1631157
1629673
1630541
1626172
1631116
2738
1629660
202691
203952
203110

In [44]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
game_ids = []

for i in nba_id:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=i,
                            season_nullable="2022-23",
                            season_type_nullable=SeasonType.regular)   
    games_dict = gamefinder.get_normalized_dict()
    games = games_dict['LeagueGameFinderResults']
    df = pd.DataFrame.from_dict(games)
    #a = df['GAME_ID'].unique().tolist()
    game_ids.append(df)


In [45]:
games = pd.concat(game_ids)

In [46]:
games['opp_abbr'] = games['MATCHUP'].str[-3:]

In [47]:
gm = games[['TEAM_ID','TEAM_NAME','GAME_DATE','GAME_ID','opp_abbr' ]]

In [48]:
start_date = datetime.datetime.strptime("2022-10-18", "%Y-%m-%d")
yest_date = (date.today()- timedelta(days = 1)).strftime("%Y-%m-%d")

In [30]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
plshots = []

for i in nba_rosters.iterrows():
    try:
        s = shotchartdetail.ShotChartDetail(context_measure_simple = "FGA",  player_id = i[1][2], team_id=i[1][0], date_from_nullable=start_date ,date_to_nullable=yest_date).get_normalized_dict()
        shots = s['Shot_Chart_Detail']
        pj = pd.DataFrame.from_dict(shots)
        pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA'].str.split('(').str[1]
        pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
        pj['SHOT_ZONE'] = pj['SHOT_ZONE_BASIC'] + ' | ' + pj['SHOT_ZONE_AREA_abbr']+ ' | ' + pj['SHOT_ZONE_RANGE']
        pj1 = pj[['GAME_ID', 'GAME_EVENT_ID','PLAYER_ID','PLAYER_NAME','TEAM_ID','TEAM_NAME','PERIOD','SHOT_ATTEMPTED_FLAG','SHOT_MADE_FLAG','SHOT_ZONE','LOC_X','LOC_Y']]
        #a = df['GAME_ID'].unique().tolist()
        plshots.append(pj1)
    except:
        pass

/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex

/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex

/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex

/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex

/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex

...

/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex

/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex

/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex

/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex

/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_51494/4163585144.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex

In [32]:
pshots = pd.concat(plshots)

In [107]:
pshots.loc[pshots['SHOT_MADE_FLAG']!=1]

,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,SHOT_ZONE,LOC_X,LOC_Y
1,0022200020,134,1630552,Jalen Johnson,1610612737,Atlanta Hawks,1,1,0,In The Paint (Non-RA) | C | Less Than 8 ft.,-69,34
2,0022200020,172,1630552,Jalen Johnson,1610612737,Atlanta Hawks,2,1,0,Left Corner 3 | L | 24+ ft.,-233,-9
4,0022200020,180,1630552,Jalen Johnson,1610612737,Atlanta Hawks,2,1,0,Above the Break 3 | LC | 24+ ft.,-189,174
6,0022200038,151,1630552,Jalen Johnson,1610612737,Atlanta Hawks,1,1,0,Left Corner 3 | L | 24+ ft.,-235,19
1,0022200005,424,1628988,Aaron Holiday,1610612737,Atlanta Hawks,3,1,0,Above the Break 3 | C | 24+ ft.,-74,232
...,...,...,...,...,...,...,...,...,...,...,...,...
23,0022200017,668,1629023,P.J. Washington,1610612766,Charlotte Hornets,4,1,0,Above the Break 3 | C | 24+ ft.,-53,257
25,0022200038,45,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,0,In The Paint (Non-RA) | C | Less Than 8 ft.,-22,49
29,0022200038,585,1629023,P.J. Washington,1610612766,Charlotte Hornets,4,1,0,Above the Break 3 | LC | 24+ ft.,-89,230
30,0022200038,652,1629023,P.J. Washington,1610612766,Charlotte Hornets,4,1,0,Above the Break 3 | RC | 24+ ft.,104,234


In [33]:
pshots.to_csv('shotchartloc.csv')

In [39]:
shotchart = pd.merge(pshots, gm, on=['GAME_ID','TEAM_ID','TEAM_NAME'], how='left')

In [40]:
shotchart

,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,SHOT_ZONE,LOC_X,LOC_Y,GAME_DATE,opp_abbr
0,0022200099,673,1626163,Frank Kaminsky,1610612737,Atlanta Hawks,4,1,0,Above the Break 3 | RC | 24+ ft.,225,109,2022-10-31,TOR
1,0022200005,486,1630552,Jalen Johnson,1610612737,Atlanta Hawks,4,1,1,Restricted Area | C | Less Than 8 ft.,13,1,2022-10-19,HOU
2,0022200020,134,1630552,Jalen Johnson,1610612737,Atlanta Hawks,1,1,0,In The Paint (Non-RA) | C | Less Than 8 ft.,-69,34,2022-10-21,ORL
3,0022200020,172,1630552,Jalen Johnson,1610612737,Atlanta Hawks,2,1,0,Left Corner 3 | L | 24+ ft.,-233,-9,2022-10-21,ORL
4,0022200020,177,1630552,Jalen Johnson,1610612737,Atlanta Hawks,2,1,1,Restricted Area | C | Less Than 8 ft.,-4,10,2022-10-21,ORL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21191,0022200111,171,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,0,In The Paint (Non-RA) | C | Less Than 8 ft.,-43,34,2022-11-02,CHI
21192,0022200111,212,1629023,P.J. Washington,1610612766,Charlotte Hornets,2,1,1,Restricted Area | C | Less Than 8 ft.,-20,12,2022-11-02,CHI
21193,0022200111,223,1629023,P.J. Washington,1610612766,Charlotte Hornets,2,1,0,Right Corner 3 | R | 24+ ft.,233,-30,2022-11-02,CHI
21194,0022200111,500,1629023,P.J. Washington,1610612766,Charlotte Hornets,3,1,0,In The Paint (Non-RA) | C | Less Than 8 ft.,-31,32,2022-11-02,CHI


In [41]:
from gspread_pandas import Spread, Client
spread = Spread('NBA_stats')
spread.df_to_sheet(shotchart, index=False,sheet='NBA_shotchart',replace=True)

In [ ]:
shotchart.to_csv('shotchartvsteam.csv')

In [ ]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import playergamelogs
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
plshots = []


s = playergamelogs.PlayerGameLogs(season_nullable='2022-23').get_normalized_dict()
games = s['PlayerGameLogs']
gl = pd.DataFrame.from_dict(games)
        #pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA'].str.split('(').str[1]
        #pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')
        #pj['SHOT_ZONE'] = pj['SHOT_ZONE_BASIC'] + ' | ' + pj['SHOT_ZONE_AREA_abbr']+ ' | ' + pj['SHOT_ZONE_RANGE']
        #pj1 = pj[['GAME_ID', 'GAME_EVENT_ID','PLAYER_ID','PLAYER_NAME','TEAM_ID','TEAM_NAME','PERIOD','SHOT_ATTEMPTED_FLAG','SHOT_MADE_FLAG','SHOT_ZONE','LOC_X','LOC_Y']]
        #a = df['GAME_ID'].unique().tolist()
        #plshots.append(pj1)

In [ ]:
gl['opp_abbr'] = gl['MATCHUP'].str[-3:]

In [ ]:
#gamelogs = gl[['GAME_DATE','GAME_ID','PLAYER_ID','PLAYER_NAME','TEAM_ID','TEAM_NAME','opp_abbr','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA','PTS','REB','AST','TOV','STL','BLK','PTS_RANK','FGA_RANK','FG3A_RANK','FTA_RANK','REB_RANK','AST_RANK']]

In [ ]:
nba_rosters.rename(columns={'TeamID':'TEAM_ID','PLAYER':'PLAYER_NAME'},inplace=True)

In [ ]:
gl

In [ ]:
nbagamelogs = pd.merge(gl, nba_rosters, on=['TEAM_ID','PLAYER_ID','PLAYER_NAME'],how='left')

In [ ]:
gamelogs = nbagamelogs[['GAME_DATE','GAME_ID','PLAYER_ID','PLAYER_NAME','POSITION','HEIGHT','WEIGHT','TEAM_ID','TEAM_NAME','opp_abbr','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA','PTS','REB','AST','TOV','STL','BLK','PTS_RANK','FGA_RANK','FG3A_RANK','FTA_RANK','REB_RANK','AST_RANK']]

In [ ]:
gamelogs

In [ ]:
from gspread_pandas import Spread, Client
spread = Spread('NBA_stats')
spread.df_to_sheet(gamelogs, index=False,sheet='NBA_gamelogs',replace=True)

In [28]:
s = shotchartdetail.ShotChartDetail(player_id = 1629023, team_id=1610612766).get_normalized_dict()
shots = s['Shot_Chart_Detail']
pj = pd.DataFrame.from_dict(shots)

In [39]:
pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA'].str.split('(').str[1]

In [42]:
pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')

/var/folders/sj/rsx4zyld6flczf6q6r20b1c80000gn/T/ipykernel_1852/2242184280.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pj['SHOT_ZONE_AREA_abbr'] = pj['SHOT_ZONE_AREA_abbr'].str.replace(")",'')


In [46]:
pj['SHOT_ZONE'] = pj['SHOT_ZONE_BASIC'] + ' | ' + pj['SHOT_ZONE_AREA_abbr']+ ' | ' + pj['SHOT_ZONE_RANGE']

In [49]:
pj1 = pj[['GAME_ID', 'GAME_EVENT_ID','PLAYER_ID','PLAYER_NAME','TEAM_ID','TEAM_NAME','PERIOD','SHOT_ATTEMPTED_FLAG','SHOT_MADE_FLAG','SHOT_ZONE']]

In [50]:
pj1

,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,SHOT_ZONE
0,0021900003,27,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,1,Above the Break 3 | RC | 24+ ft.
1,0021900003,60,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,1,Left Corner 3 | L | 24+ ft.
2,0021900003,64,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,1,Above the Break 3 | LC | 24+ ft.
3,0021900003,308,1629023,P.J. Washington,1610612766,Charlotte Hornets,2,1,1,Left Corner 3 | L | 24+ ft.
4,0021900003,321,1629023,P.J. Washington,1610612766,Charlotte Hornets,2,1,1,Left Corner 3 | L | 24+ ft.
...,...,...,...,...,...,...,...,...,...,...
815,0022101208,538,1629023,P.J. Washington,1610612766,Charlotte Hornets,4,1,1,Restricted Area | C | Less Than 8 ft.
816,0022101217,30,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,1,Restricted Area | C | Less Than 8 ft.
817,0022101217,67,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,1,Restricted Area | C | Less Than 8 ft.
818,0022101217,73,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,1,Restricted Area | C | Less Than 8 ft.


In [69]:
s = leaguedashplayershotlocations.LeagueDashPlayerShotLocations().get_data_frames()[0]
#shots = s['Shot_Chart_Detail
#pj = pd.DataFrame.from_dict(shots)
s

Empty DataFrame
Columns: [(, PLAYER_ID), (, PLAYER_NAME), (, TEAM_ID), (, TEAM_ABBREVIATION), (, AGE), (, NICKNAME), (Restricted Area, FGM), (Restricted Area, FGA), (Restricted Area, FG_PCT), (In The Paint (Non-RA), FGM), (In The Paint (Non-RA), FGA), (In The Paint (Non-RA), FG_PCT), (Mid-Range, FGM), (Mid-Range, FGA), (Mid-Range, FG_PCT), (Left Corner 3, FGM), (Left Corner 3, FGA), (Left Corner 3, FG_PCT), (Right Corner 3, FGM), (Right Corner 3, FGA), (Right Corner 3, FG_PCT), (Above the Break 3, FGM), (Above the Break 3, FGA), (Above the Break 3, FG_PCT), (Backcourt, FGM), (Backcourt, FGA), (Backcourt, FG_PCT), (Corner 3, FGM), (Corner 3, FGA), (Corner 3, FG_PCT)]
Index: []

In [19]:
#year_list = range(2020,2021)
#games_dict = gamefinder.get_normalized_dict()
#games = games_dict['LeagueGameLog']
#games
#pd.DataFrame.from_dict(games)

In [38]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
game_ids = []

for i in nba_id:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=i,
                            season_nullable="2022-23",
                            season_type_nullable=SeasonType.regular)   
    games_dict = gamefinder.get_normalized_dict()
    games = games_dict['LeagueGameFinderResults']
    df = pd.DataFrame.from_dict(games)
    #a = df['GAME_ID'].unique().tolist()
    game_ids.append(df)




In [39]:
games = pd.concat(game_ids)

In [40]:
games['opp_abbr'] = games['MATCHUP'].str[-3:]
#games['tm1'] = games['MATCHUP'].str[:3]

In [34]:
#games.loc[games['TEAM_ABBREVIATION'] == games['tm1'],'opp_abbr']= games['tm2']
#games.loc[games['TEAM_ABBREVIATION'] != games['tm1'],'opp_abbr']= games['tm1']

In [41]:
gm = games[['TEAM_ID','TEAM_NAME','GAME_DATE','GAME_ID','opp_abbr' ]]

In [96]:
pjgms = pd.merge(pj1, gm, on=['GAME_ID','TEAM_ID','TEAM_NAME'],how='left')

In [97]:
pjgms

,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,SHOT_ZONE,GAME_DATE,opp_abbr
0,0021900003,27,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,1,Above the Break 3 | RC | 24+ ft.,NaN,NaN
1,0021900003,60,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,1,Left Corner 3 | L | 24+ ft.,NaN,NaN
2,0021900003,64,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,1,Above the Break 3 | LC | 24+ ft.,NaN,NaN
3,0021900003,308,1629023,P.J. Washington,1610612766,Charlotte Hornets,2,1,1,Left Corner 3 | L | 24+ ft.,NaN,NaN
4,0021900003,321,1629023,P.J. Washington,1610612766,Charlotte Hornets,2,1,1,Left Corner 3 | L | 24+ ft.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
815,0022101208,538,1629023,P.J. Washington,1610612766,Charlotte Hornets,4,1,1,Restricted Area | C | Less Than 8 ft.,2022-04-08,CHI
816,0022101217,30,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,1,Restricted Area | C | Less Than 8 ft.,2022-04-10,WAS
817,0022101217,67,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,1,Restricted Area | C | Less Than 8 ft.,2022-04-10,WAS
818,0022101217,73,1629023,P.J. Washington,1610612766,Charlotte Hornets,1,1,1,Restricted Area | C | Less Than 8 ft.,2022-04-10,WAS


In [ ]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
game_ids = []

for i in nba_id:
    gamefinder = shotchartdetail.ShotChartDetail(team_id_nullable=i,
                            season_nullable="2021-22",
                            season_type_nullable=SeasonType.regular)   
    games_dict = gamefinder.get_normalized_dict()
    games = games_dict['LeagueGameFinderResults']
    df = pd.DataFrame.from_dict(games)
    #a = df['GAME_ID'].unique().tolist()
    game_ids.append(df)


In [78]:
games['GAME_DATE'].sort_values()

7    2022-07-02
7    2022-07-02
7    2022-07-02
7    2022-07-02
6    2022-07-03
        ...    
0    2022-07-17
0    2022-07-17
0    2022-07-17
0    2022-07-17
0    2022-07-17
Name: GAME_DATE, Length: 177, dtype: object